<a href="https://colab.research.google.com/github/shghatge/pointnet/blob/master/Feature1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import nltk 
from nltk import word_tokenize
from nltk.util import ngrams
import pickle
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from nltk.stem.porter import PorterStemmer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def dump_to_file(obj, file):
    f = open(file,"wb")
    pickle.dump(obj,f)
    f.close()
def get_obj(file):
    with open(file, 'rb') as handle:
        obj = pickle.load(handle)
    return obj
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems
  

In [21]:
trainFile = pd.read_csv('TopTenTrain.csv', low_memory=False)
testFile = pd.read_csv('TopTenTest.csv', low_memory=False)
print('Data Read')

Data Read


In [22]:
print(trainFile["cleanText"][108])
print(trainFile["text"][108])
print(trainFile["fillers"][108])

Our state doesnt yet say that 
Our state doesn't yet, <laughter> say that  /
None


In [23]:
train_corpus_file = 'train_corpus.pkl'
test_corpus_file = 'test_corpus.pkl'
train_corpus = get_obj(train_corpus_file)
test_corpus = get_obj(test_corpus_file)
print(train_corpus[:10])

['Does say something', 'I think usually', 'Okay', 'Well', 'Okay', 'I', 'It seemed like', 'might', 'I guess', 'Okay']


In [0]:
pos_dict = dict()
labels_dict = dict()
#nltk.download('averaged_perceptron_tagger')
for index , row in trainFile.iterrows():
    tokens = nltk.word_tokenize(row['cleanText'])
    tagged = nltk.pos_tag(tokens)
    l=[]
    for tag in tagged:
  #    l.append(tag[0])
      l.append(tag[1])
    pos_dict[index] = " ".join(l)
    labels_dict[index] = row['act_tag']

In [0]:
pos_dict_test = dict()
labels_dict_test = dict()
#nltk.download('averaged_perceptron_tagger')
for index , row in testFile.iterrows():
    tokens = nltk.word_tokenize(row['cleanText'])
    tagged = nltk.pos_tag(tokens)
    l=[]
    for tag in tagged:
#       l.append(tag[0])
      l.append(tag[1])
    pos_dict_test[index] = " ".join(l)
    labels_dict_test[index] = row['act_tag']

In [0]:
corpus=list(pos_dict.values())

In [41]:
vectorizer = CountVectorizer(tokenizer = tokenize)
vectorizer.fit(corpus)
vocab = list(vectorizer.vocabulary_.keys()) +list(vectorizer.stop_words_)
print(vocab)

['vbz', 'prp', 'vb', 'nn', 'vbp', 'rb', 'vbd', 'in', 'cc', 'md', '$', 'dt', 'cd', 'jj', 'vbg', 'to', 'wp', 'nnp', 'uh', 'wrb', 'vbn', 'jjr', 'ex', 'rp', 'pdt', 'wdt', 'rbr', 'fw', 'po', "''", 'sym']


In [34]:
le = preprocessing.LabelEncoder()
le.fit(vocab)

LabelEncoder()

In [0]:
features_rows = dict()
for index , row in trainFile.iterrows():
  features_rows[index] = le.transform(tokenize(pos_dict[index].lower()))

In [0]:
features = list(features_rows.values())
padded = pad_sequences(features, maxlen=100)

In [0]:
import bisect
test_rows = dict()
i=0
le_classes = le.classes_.tolist()
bisect.insort_left(le_classes, 'otherClass')
le.classes_ = le_classes
for index, row in testFile.iterrows():
  test = ['otherClass' if s not in le.classes_ else s for s in tokenize(row['cleanText'].lower())]
  test_rows[index] = le.transform(test)
X_test = pad_sequences(list(test_rows.values()), maxlen=100)

In [45]:
enc = preprocessing.LabelEncoder()
enc.fit(np.array([*labels_dict.values()]))

LabelEncoder()

In [0]:
y = enc.transform(np.array([*labels_dict.values()]))
y_test = enc.transform(np.array([*labels_dict_test.values()]))

In [0]:
model = OneVsRestClassifier(RandomForestClassifier(n_estimators=100)).fit(padded, y)

In [0]:
preds = model.predict(X_test)

In [0]:
target_names = enc.classes_
print(classification_report(y_test, preds, target_names=target_names))